In [ ]:
import pyam
import nomenclature

In [ ]:
df_1 = pyam.IamDataFrame("source/7/Chapter7_Fig1_final_data.xlsx")
df_2 = pyam.IamDataFrame("source/7/Chapter7_Fig2_final_data.xlsx")
df_4 = pyam.IamDataFrame("source/7/Chapter7_Fig4_final_data.xlsx")

In [ ]:
df = pyam.concat(
    [df_1, df_2, df_4]
)

In [ ]:
df_2.variable

In [ ]:
dsd = nomenclature.DataStructureDefinition("../definitions/")

In [ ]:
mapping = dict(
    [
        (i, (
                i
                .replace("Direct", "Conventional CDR")
                .replace("AR6 climate diagnostics", "AR6 Climate Diagnostics")
                .replace("Gross Emissions", "Gross Emissions|CO2 [Reductions relative to 2020]")
                .replace("CO2|CO2", "CO2")
                .replace("AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land|Conventional CDR", "Carbon Removal|Land")
            )
        )
        for i in df.variable
    ]
)

df.rename(variable=mapping, inplace=True)

In [ ]:
df.rename(
    model={"SoCDR_v2": "State of CDR (2024) - Paris-Consistent Pathways"},
    inplace=True
)

In [ ]:
dsd.validate(df)

In [ ]:
df.to_excel("Chapter8_all_data.xlsx")

In [ ]:
df

In [ ]:
df.to_ixmp4("socdr-dev")

In [ ]:
import ixmp4

In [ ]:
platform = ixmp4.Platform("socdr-dev")

In [ ]:
platform.regions.create("World", hierarchy="common")

In [ ]:
platform.units.create("Gt CO2/yr")

In [ ]:
df.set_meta(meta="Chapter 8", name="Chapter")

In [ ]:
pyam.iiasa.platforms()

In [ ]:
df